In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm

#https://www.ncdc.noaa.gov/cdo-web/webservices/v2#data

Token = 'IQETwdnSgIvPGJcWckJewaPKmTupEAou'
StationID = 'USW00024230' # Redmond

In [9]:
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets/GHCND',headers={'token':Token})

In [10]:
d = json.loads(r.text)

In [11]:
d

{'mindate': '1763-01-01',
 'maxdate': '2020-06-01',
 'name': 'Daily Summaries',
 'datacoverage': 1,
 'id': 'GHCND'}

In [43]:
apicall = r'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:'+StationID+'&units=standard&startdate=1995-01-01&enddate=2019-05-31'

req = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:'+'USC00010008'+'&units=standard&startdate=1995-01-01&enddate=2004-05-31',headers={'token':Token})

In [44]:
js = json.loads(req.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
req

<Response [400]>

In [29]:
apicall.replace('&amp;','&')

'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:USW00024230&units=standard&startdate=1995-01-01&enddate=2019-05-31'

In [84]:
api2 = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:USW00024230&units=standard&startdate=1995-05-01&enddate=1995-10-31&limit=1000'

print(api2)

req2 = requests.get(api2,headers={'token':Token})
d2 = json.loads(req2.text)
d2

https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:USW00024230&units=standard&startdate=1995-05-01&enddate=1995-10-31&limit=1000


{'metadata': {'resultset': {'offset': 1, 'count': 978, 'limit': 1000}},
 'results': [{'date': '1995-05-01T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,2400',
   'value': 0.25},
  {'date': '1995-05-01T00:00:00',
   'datatype': 'SNOW',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,',
   'value': 0.0},
  {'date': '1995-05-01T00:00:00',
   'datatype': 'SNWD',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,',
   'value': 0.0},
  {'date': '1995-05-01T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,2400',
   'value': 54.0},
  {'date': '1995-05-01T00:00:00',
   'datatype': 'TMIN',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,2400',
   'value': 43.0},
  {'date': '1995-05-01T00:00:00',
   'datatype': 'WT16',
   'station': 'GHCND:USW00024230',
   'attributes': ',,0,',
   'value': 1},
  {'date': '1995-05-02T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:USW00024230',
  

In [59]:
req2.text

'{"metadata":{"resultset":{"offset":1,"count":978,"limit":25}},"results":[{"date":"1995-05-01T00:00:00","datatype":"PRCP","station":"GHCND:USW00024230","attributes":",,0,2400","value":0.25},{"date":"1995-05-01T00:00:00","datatype":"SNOW","station":"GHCND:USW00024230","attributes":",,0,","value":0.0},{"date":"1995-05-01T00:00:00","datatype":"SNWD","station":"GHCND:USW00024230","attributes":",,0,","value":0.0},{"date":"1995-05-01T00:00:00","datatype":"TMAX","station":"GHCND:USW00024230","attributes":",,0,2400","value":54.0},{"date":"1995-05-01T00:00:00","datatype":"TMIN","station":"GHCND:USW00024230","attributes":",,0,2400","value":43.0},{"date":"1995-05-01T00:00:00","datatype":"WT16","station":"GHCND:USW00024230","attributes":",,0,","value":1},{"date":"1995-05-02T00:00:00","datatype":"PRCP","station":"GHCND:USW00024230","attributes":",,0,2400","value":0.0},{"date":"1995-05-02T00:00:00","datatype":"SNOW","station":"GHCND:USW00024230","attributes":",,0,","value":0.0},{"date":"1995-05-02T0

In [62]:
def get_weather_multiyear(station_id, start_date, end_date, token, limit=1000, units='standard', datasetid='GHCND',**kwargs):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if (end_date - start_date).days <= 365:
        weather_dict_list, results_dict = get_weather_between_dates(station_id, start_date, end_date, token, limit=limit, units=units, datasetid=datasetid, **kwargs)
    else:
        chunks = (end_date - start_date).days // 300 + 1 # number of 300 day chunks
        leftover = (end_date - start_date).days % 300 # leftover days after chunks-1 chunks
        
        weather_dict_list = []
        
        if leftover != 0:
            start_dates = [start_date] + [start_date + pd.to_timedelta(i * 300 + 1, unit='d') for i in range(1,chunks)]
            end_dates = [start_date + pd.to_timedelta(j * 300, unit='d') for j in range(1,chunks)] + [end_date]
        else:
            start_dates = [start_date] + [start_date + pd.to_timedelta(i * 300 + 1, unit='d') for i in range(1,chunks-1)]
            end_dates = [start_date + pd.to_timedelta(j * 300, unit='d') for j in range(1,chunks-1)] + [end_date]
        
        for sd, ed in zip(tqdm(start_dates,desc='Date Chunks'), end_dates):
            #print(sd,ed)
            wdl, results_dict = get_weather_between_dates(station_id, sd, ed, token, limit=limit, units=units, datasetid=datasetid, **kwargs)
            #print(len(wdl))
            weather_dict_list = weather_dict_list + wdl
    return weather_dict_list, results_dict

In [63]:
big1, res1 = get_weather_multiyear(StationID, '1990-01-01', '1993-04-01', Token, limit=1000)

In [57]:
len(big1),res1

(7047, {'resultset': {'offset': 1001, 'count': 1941, 'limit': 1000}})

In [40]:
start_date = pd.to_datetime('1995-01-01')
end_date = pd.to_datetime('1997-06-18')

In [41]:
chunks = (end_date - start_date).days // 300 + 1
leftover = (end_date - start_date).days % 300
print(chunks,leftover)
#str(start_date.date())

3 299


In [44]:
start_dates = [start_date] + [start_date + pd.to_timedelta(i * 300 + 1, unit='d') for i in range(1,chunks)]
end_dates = [start_date + pd.to_timedelta(j * 300, unit='d') for j in range(1,chunks)] + [end_date]

for sd, ed in zip(start_dates, end_dates):
    print(sd,ed)

1995-01-01 00:00:00 1995-10-28 00:00:00
1995-10-29 00:00:00 1996-08-23 00:00:00
1996-08-24 00:00:00 1997-06-18 00:00:00


In [58]:
def get_weather_between_dates(station_id, start_date, end_date, token, limit=1000, units='standard', datasetid='GHCND',**kwargs):
    '''Makes an API call to NOAA from station = station_id, between start_date and end_date, with
    token = token, and addition **kwargs (such as datasetid='GHCND', units='standard')
    '''
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    if (end_date - start_date).days > 365:
        return print("Error, too long of a gap")
    
    apicall = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?stationid=GHCND:'+station_id+\
                '&datasetid='+datasetid+'&startdate='+str(start_date.date())+'&enddate='+str(end_date.date())+\
                '&units='+units+'&limit='+str(limit)
    
    if kwargs.__len__() != 0:
        for key, val in kwargs.items():
            apicall = apicall+'&'+str(key)+'='+str(val)
            
    #print(apicall)
    request = requests.get(apicall,headers={'token':Token})
    
    js = json.loads(request.text)
    
    count = js['metadata']['resultset']['count']
    
    fullresults = js['results']
    
    if count > limit:
        reps = count // limit
        
        if count % limit != 0:
            for iteration in range(reps):
                # print('offset=',iteration*limit)
                request = requests.get(apicall + '&offset=' + str((iteration + 1) * limit + 1),headers={'token':Token})
                js = json.loads(request.text)
                fullresults = fullresults + js['results']
        else:
            for iteration in range(reps-1):
                # print('offset=',iteration*limit)
                request = requests.get(apicall + '&offset=' + str((iteration + 1) * limit + 1),headers={'token':Token})
                js = json.loads(request.text)
                fullresults = fullresults + js['results']
    #else:
    #    fullresults = fullresults + js['results']
    # Check to see if ['metadata']['resultset']['count'] > ['metadata']['resultset']['limit']
    # if it is, then iterate c//l + 1 times, calling and building the df
    
    return fullresults, js['metadata']
    

In [46]:
gq1, res1 = get_weather_between_dates(StationID, '1990-01-01', '1993-04-01', Token, limit=1000)

Error, too long of a gap


TypeError: cannot unpack non-iterable NoneType object

In [141]:
c = gq1['metadata']['resultset']['count']

In [140]:
l = gq1['metadata']['resultset']['limit']

In [208]:
c = 624
l = 12

In [209]:
c%l

0

In [146]:
c - c//l*l

9

In [148]:
c//l + 1

42

In [149]:
gqa = gq1
gqb = gq1

In [153]:
gqa['results'][-1]

{'date': '1990-01-02T00:00:00',
 'datatype': 'WT01',
 'station': 'GHCND:USW00024230',
 'attributes': ',,0,',
 'value': 1}

In [157]:
gqc = []


In [158]:
gqc = gqc + gqa['results']

In [160]:
gqc = gqc + gqb['results']

In [162]:
len(gqc)

30

In [212]:
len(gq1)

624

In [213]:
res1, len(gq1)
    

({'resultset': {'offset': 613, 'count': 624, 'limit': 12}}, 624)

In [214]:
gq1[10:14]

[{'date': '1990-01-02T00:00:00',
  'datatype': 'SNOW',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,',
  'value': 0.7},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'SNWD',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,',
  'value': 0.0},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'TMAX',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,2400',
  'value': 41.0},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'TMIN',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,2400',
  'value': 18.0}]

In [215]:
gq2[10:14]

[{'date': '1990-01-02T00:00:00',
  'datatype': 'SNOW',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,',
  'value': 0.7},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'SNWD',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,',
  'value': 0.0},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'TMAX',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,2400',
  'value': 41.0},
 {'date': '1990-01-02T00:00:00',
  'datatype': 'TMIN',
  'station': 'GHCND:USW00024230',
  'attributes': ',,0,2400',
  'value': 18.0}]

In [77]:
index = np.unique([str(pd.to_datetime(big1[i]['date']).date()) for i in range(len(big1))])

In [78]:
df = pd.DataFrame(None, index=index)

In [79]:
df

""
1990-01-01
1990-01-02
1990-01-03
1990-01-04
1990-01-05
...
1993-03-28
1993-03-29
1993-03-30
1993-03-31


In [82]:
unique_datatypes = np.unique([str(big1[i]['datatype']) for i in range(len(big1))])
unique_datatypes

array(['AWND', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'WT01', 'WT02',
       'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT14',
       'WT15', 'WT16', 'WT17', 'WT18'], dtype='<U4')

In [83]:
for dtype in unique_datatypes:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-83-0229d9c6ba58>, line 2)

In [103]:
df = pd.DataFrame(big1)
df['date'] = pd.to_datetime(df['date'])

In [106]:
df = df.pivot(index='date',columns='datatype',values='value')

In [102]:
df['date'] = pd.to_datetime(df['date'])
df['date']

0      1990-01-01
1      1990-01-01
2      1990-01-01
3      1990-01-01
4      1990-01-01
          ...    
7042   1993-04-01
7043   1993-04-01
7044   1993-04-01
7045   1993-04-01
7046   1993-04-01
Name: date, Length: 7047, dtype: datetime64[ns]

In [109]:
keeps = ['AWND','PRCP','SNOW','SNWD','TMAX','TMIN']
df = df[keeps]

In [111]:
df.describe()

datatype,AWND,PRCP,SNOW,SNWD,TMAX,TMIN
count,1041.000000,1058.000000,1058.000000,1059.000000,1059.000000,1058.000000
mean,6.064457,0.022212,0.073724,0.418319,59.084986,32.001890
std,2.805768,0.066871,0.469777,1.781995,18.256271,12.178893
min,0.000000,0.000000,0.000000,0.000000,3.000000,-24.000000
25%,4.300000,0.000000,0.000000,0.000000,46.000000,25.000000
50%,5.800000,0.000000,0.000000,0.000000,57.000000,32.000000
75%,7.600000,0.000000,0.000000,0.000000,72.000000,40.000000
max,17.200000,0.560000,9.000000,14.000000,100.000000,64.000000
